<a href="https://colab.research.google.com/github/Gnemasime/Music-Generation/blob/main/music_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install music21
!pip install tensorflow
!pip install numpy


In [ ]:
import music21
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.utils import to_categorical


In [ ]:
from google.colab import files
import music21

# Upload files
uploaded = files.upload()

# Parse the uploaded files
notes = []

for file in uploaded.keys():
    midi = music21.converter.parse(file)
    notes_to_parse = None
    parts = music21.instrument.partitionByInstrument(midi)
    if parts:  # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else:  # file has notes in flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, music21.note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, music21.chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))


Saving Pendulum - Witchcraft  (midi by Carlo Prato) (www.cprato.com).mid to Pendulum - Witchcraft  (midi by Carlo Prato) (www.cprato.com).mid
Saving Porter Robinson - Sad Machine  (midi by Carlo Prato) (www.cprato.com).mid to Porter Robinson - Sad Machine  (midi by Carlo Prato) (www.cprato.com).mid
Saving San Holo - Light  (midi by Carlo Prato) (www.cprato.com).mid to San Holo - Light  (midi by Carlo Prato) (www.cprato.com).mid


In [ ]:
# Get all pitch names
pitches = sorted(set(item for item in notes))
n_vocab = len(pitches)

# Create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitches))

sequence_length = 100
network_input = []
network_output = []

# Create input sequences and the corresponding outputs
for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in seq_in])
    network_output.append(note_to_int[seq_out])

n_patterns = len(network_input)

# Reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)


In [ ]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.fit(network_input, network_output, epochs=200, batch_size=64)


Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - loss: 3.4994
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 3.3046
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 3.1038
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 3.1968
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 3.1190
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.0945
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 3.1220
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 3.1905
Epoch 9/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.0840
Epoch 10/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - loss: 3.1028
Epoch 11/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 3.0904
Epoch 12/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - loss: 3.0911
Epoch 13/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 3.1013
Epoch 14/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 3.1224
Epoch 15/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - loss: 3.1302
Epoch 16/200
7/7 ━━━━━━━━━━━━━━━━━

In [ ]:
import random

start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]
prediction_output = []

# Generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)
    int_to_note = dict((number, note) for number, note in enumerate(pitches))
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]

# Convert the output to a MIDI file
offset = 0
output_notes = []

for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():  # Chord
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = music21.note.Note(int(current_note))
            new_note.storedInstrument = music21.instrument.Piano()
            notes.append(new_note)
        new_chord = music21.chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:  # Note
        new_note = music21.note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = music21.instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5

midi_stream = music21.stream.Stream(output_notes)
midi_stream.write('midi', fp='output.mid')


'output.mid'

In [ ]:
from google.colab import files
files.download('output.mid')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>